## Lab 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

## First, looking at `accounts.py`

Let's review some python code made mostly by a hard-working Engineering Team.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=6730.273599999999, strategy='Monitor market closely for opportunities based on volatility and news.', holdings={'DIS': 20, 'AMZN': 6}, transactions=[20 shares of DIS at 105.41040000000001 each., 3 shares of AMZN at 193.5864 each., 3 shares of AMZN at 193.5864 each.], portfolio_value_time_series=[('2025-05-08 19:54:56', 10000.0), ('2025-05-08 19:55:04', 10000.0), ('2025-05-08 19:55:57', 9995.792), ('2025-05-08 19:57:26', 9991.792), ('2025-05-09 22:30:24', 10010.2328), ('2025-05-09 22:30:26', 10010.2328), ('2025-05-09 22:33:16', 10010.2328), ('2025-05-10 21:28:53', 10009.0736), ('2025-05-10 21:28:58', 10009.0736), ('2025-05-10 21:31:11', 10009.0736)])

In [4]:
account.buy_shares("AMZN", 1, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 6536.6871999999985, "strategy": "Monitor market closely for opportunities based on volatility and news.", "holdings": {"DIS": 20, "AMZN": 7}, "transactions": [{"symbol": "DIS", "quantity": 20, "price": 105.41040000000001, "timestamp": "2025-05-08 19:55:57", "rationale": "Positive market performance and strong day change of 3.04%."}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-09 22:30:24", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-10 21:28:53", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 193.5864, "timestamp": "2025-05-11 13:09:24", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-08 19:54:56", 10000.0], ["2025-05-08 19:55:04", 10000.0], ["2025-05-08 19:55:57", 9995.792], ["2025-05-08 19

In [5]:
account.report()

'{"name": "ed", "balance": 6536.6871999999985, "strategy": "Monitor market closely for opportunities based on volatility and news.", "holdings": {"DIS": 20, "AMZN": 7}, "transactions": [{"symbol": "DIS", "quantity": 20, "price": 105.41040000000001, "timestamp": "2025-05-08 19:55:57", "rationale": "Positive market performance and strong day change of 3.04%."}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-09 22:30:24", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-10 21:28:53", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 193.5864, "timestamp": "2025-05-11 13:09:24", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-08 19:54:56", 10000.0], ["2025-05-08 19:55:04", 10000.0], ["2025-05-08 19:55:57", 9995.792], ["2025-05-08 19:57:26", 9991.792], ["2025-0

In [6]:
account.list_transactions()

[{'symbol': 'DIS',
  'quantity': 20,
  'price': 105.41040000000001,
  'timestamp': '2025-05-08 19:55:57',
  'rationale': 'Positive market performance and strong day change of 3.04%.'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 193.5864,
  'timestamp': '2025-05-09 22:30:24',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 193.5864,
  'timestamp': '2025-05-10 21:28:53',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 1,
  'price': 193.5864,
  'timestamp': '2025-05-11 13:09:24',
  'rationale': 'Because this bookstore website looks promising'}]

## Now we make an MCP server: `accounts_server.py`

In [9]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [10]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [11]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your account balance is **$6,536.69**.

As for your holdings, you have:
- **20 shares** of Disney (DIS)
- **7 shares** of Amazon (AMZN)

OpenAI Agents SDK takes care of the MCP Client, but we can also make one ourselves

## See `accounts_client.py` for our MCP Client

In [12]:
from accounts_client import read_accounts_resource

context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 6536.6871999999985, "strategy": "Monitor market closely for opportunities based on volatility and news.", "holdings": {"DIS": 20, "AMZN": 7}, "transactions": [{"symbol": "DIS", "quantity": 20, "price": 105.41040000000001, "timestamp": "2025-05-08 19:55:57", "rationale": "Positive market performance and strong day change of 3.04%."}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-09 22:30:24", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 193.5864, "timestamp": "2025-05-10 21:28:53", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 193.5864, "timestamp": "2025-05-11 13:09:24", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-08 19:54:56", 10000.0], ["2025-05-08 19:55:04", 10000.0], ["2025-05-08 19:55:57", 9995.792], ["2025-05-08 19:57:26", 9991.792], ["2025-05

## And that's how to create MCP Servers and Clients!